# APIs usage example

In [1]:
#! pip install restmagic
#! docker-compose up -d walrus

walrus = 'http://127.0.0.1:31415'

%load_ext restmagic
%rest_root --timeout 30 $walrus

import random
import string

Requests defaults are set.


## Exploring Walrus HTTP API

### Check SUI gas

In [2]:
!docker-compose exec walrus sui client gas

[warn] Client/Server api version mismatch, client api version : 1.32.3, server api version : 1.32.1
╭────────────────────────────────────────────────────────────────────┬────────────────────┬──────────────────╮
│ gasCoinId                                                          │ mistBalance (MIST) │ suiBalance (SUI) │
├────────────────────────────────────────────────────────────────────┼────────────────────┼──────────────────┤
│ 0x53e04f789b22455c23e483930ac3dc1fd7dfb65a60fc28b5784006d9be6d597b │ 869321664          │ 0.86             │
╰────────────────────────────────────────────────────────────────────┴────────────────────┴──────────────────╯


In [3]:
text = ''.join(random.choice(string.ascii_lowercase) for i in range(128))
text

'opvvoqcjsrsskrqyxnywnxgqnkqkzibuhjhkllkygiibykymrtkjwdpxehpetupqmizwvqihqatbpdsyvjwmgfbzfwwmwhehbnjukywfukpbbopbplgzormsqcympmfr'

### Upload a new data

In [4]:
%%rest PUT $walrus/v1/store

$text

{
  "newlyCreated": {
    "blobObject": {
      "id": "0x267ffbcc0bf63723cb2df50d4d5f23ecec2b0debb11931c0ef1ce68a22fb2069",
      "storedEpoch": 0,
      "blobId": "t7apjBWR1Lid4m4f5NA4nyol98gxGDZ-keofkKu8_7Y",
      "size": 128,
      "erasureCodeType": "RedStuff",
      "certifiedEpoch": 0,
      "storage": {
        "id": "0xf4ee2f23fc94c52efb401569c1db065f3e10b2dffd9e3420dd4169b175b64ccf",
        "startEpoch": 0,
        "endEpoch": 1,
        "storageSize": 65023000
      }
    },
    "encodedSize": 65023000,
    "cost": 3175000
  }
}

<Response [200]>

### Repeat request with same data

In [5]:
%%rest PUT $walrus/v1/store

$text

{
  "alreadyCertified": {
    "blobId": "t7apjBWR1Lid4m4f5NA4nyol98gxGDZ-keofkKu8_7Y",
    "event": {
      "txDigest": "37o6gU7yqWYdWD62nBhiFfX86RXBndMH2vWFdeUU7PvA",
      "eventSeq": "0"
    },
    "endEpoch": 1
  }
}

<Response [200]>

In [6]:
r = _
blob_id = r.json()["alreadyCertified"]["blobId"]
blob_id

't7apjBWR1Lid4m4f5NA4nyol98gxGDZ-keofkKu8_7Y'

### Read the data

In [7]:
%rest GET $walrus/v1/$blob_id

opvvoqcjsrsskrqyxnywnxgqnkqkzibuhjhkllkygiibykymrtkjwdpxehpetupqmizwvqihqatbpdsyvjwmgfbzfwwmwhehbnjukywfukpbbopbplgzormsqcympmfr

<Response [200]>

In [8]:
assert _.text == text

## Using the service API

### Upload encrypted message

In [9]:
%%rest POST http://127.0.0.1/secret
Content-Type: text/plain;charset=UTF-8
    
{
    "expiration":3600,
    "message":"-----BEGIN PGP MESSAGE-----\n\nwy4ECQMIE8LMYHdCKuLgDbn6q+hCfizvxRJULGLz/xj0Dqc/MGJI0E2nSbT4\nXIL70joBdGho0I9p/N8qdpWSA+44A47G4pD/mWZVK8dwGO2ILt32vaepu7ng\nY/3W3gKi3+NNpUsmW3xmpeSs\n=9TMv\n-----END PGP MESSAGE-----\n",
    "one_time":true
}

{"message":"22212f2a-81d5-44ae-abee-ae3858b077df"}

<Response [200]>

In [10]:
# Service returns the ID of stored message
storage_id = _.json()["message"]
storage_id

'22212f2a-81d5-44ae-abee-ae3858b077df'

### Redis

Redis stores the mapping of message ID -> Walrus blob ID:

In [11]:
!docker-compose exec -it redis redis-cli get $storage_id

"{\"expiration\":3600,\"message\":\"KoTGzKKLnxPUGU_ZGQsfrEB8veGQMqSZAtNljD3Ijq8\",\"one_time\":true}"


### Walrus

Corresponding blob in the Walrus:

In [12]:
%rest GET $walrus/v1/KoTGzKKLnxPUGU_ZGQsfrEB8veGQMqSZAtNljD3Ijq8

{"expiration":3600,"message":"-----BEGIN PGP MESSAGE-----\n\nwy4ECQMIE8LMYHdCKuLgDbn6q+hCfizvxRJULGLz/xj0Dqc/MGJI0E2nSbT4\nXIL70joBdGho0I9p/N8qdpWSA+44A47G4pD/mWZVK8dwGO2ILt32vaepu7ng\nY/3W3gKi3+NNpUsmW3xmpeSs\n=9TMv\n-----END PGP MESSAGE-----\n","one_time":true}

<Response [200]>

### Download encrypted message 

Client receives encrypted data, when using the service API:

In [13]:
%rest GET http://127.0.0.1/secret/$storage_id

{"expiration":3600,"message":"-----BEGIN PGP MESSAGE-----\n\nwy4ECQMIE8LMYHdCKuLgDbn6q+hCfizvxRJULGLz/xj0Dqc/MGJI0E2nSbT4\nXIL70joBdGho0I9p/N8qdpWSA+44A47G4pD/mWZVK8dwGO2ILt32vaepu7ng\nY/3W3gKi3+NNpUsmW3xmpeSs\n=9TMv\n-----END PGP MESSAGE-----\n","one_time":true}

<Response [200]>